# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [2]:
import pandas as pd

df = pd.read_csv('KNMI_20181231.txt.tsv', sep='\t')

df.rename(columns = {'# SOURCE: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)':'values'}, inplace = True)
#df = df[df['values'].str.contains('270')]
df = pd.DataFrame([ x.split(',') for x in df['values'].tolist() ])
df.rename(columns = {0: "station", 1: "Date", 2 : "Tmean", 3: "Tmin", 4: "Tmax" } ,inplace = True )
df = df[~df["station"].str.startswith('#')]
df['station'] = df['station'].astype("int")
df = df[["station","Date","Tmean","Tmin","Tmax"]]
df = df[df["station"] == 270]


In [3]:
df.head()

,station,Date,Tmean,Tmin,Tmax
97705,270,20000101,42,-4,79
97706,270,20000102,55,33,74
97707,270,20000103,74,49,89
97708,270,20000104,46,22,75
97709,270,20000105,41,14,56


<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [4]:
import numpy as np
#replace cells with spaces to NaN
df['Tmean'] = df['Tmean'].str.strip()
df['Tmin'] = df['Tmin'].str.strip()
df['Tmax'] =df['Tmax'].str.strip()
df = df.replace('',np.nan)
#change data formats && #remove leap year
df = df [~df["Date"].str.contains('-02-29')]
df['Date'] = df['Date'].str.slice(0, 4) + "-" + df['Date'].str.slice(4, 6) +  "-" + df['Date'].str.slice(6, 8) 
df["Date"] = pd.to_datetime(df["Date"])
#change temperatures to celcius degrees
df['Tmean'] = df['Tmean'].astype(float)
df['Tmin'] = df['Tmin'].astype(float)
df['Tmax'] = df['Tmax'].astype(float)


df['Tmean'] = [ x/10 for x in df['Tmean'].tolist() ]
df['Tmin'] = [ x/10 for x in df['Tmin'].tolist() ]
df['Tmax'] = [ x/10 for x in df['Tmax'].tolist() ]

df = df.set_index("Date")


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [5]:
#Test your outcome
#write code to check if you have done the above
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6940 entries, 2000-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  6940 non-null   int32  
 1   Tmean    6940 non-null   float64
 2   Tmin     6940 non-null   float64
 3   Tmax     6940 non-null   float64
dtypes: float64(3), int32(1)
memory usage: 244.0 KB


### Expected outcome

In [6]:
df

,station,Tmean,Tmin,Tmax
Date,,,,
2000-01-01,270,4.2,-0.4,7.9
2000-01-02,270,5.5,3.3,7.4
2000-01-03,270,7.4,4.9,8.9
2000-01-04,270,4.6,2.2,7.5
2000-01-05,270,4.1,1.4,5.6
...,...,...,...,...
2018-12-27,270,5.7,5.3,6.2
2018-12-28,270,7.1,5.8,8.1
2018-12-29,270,8.5,6.9,10.2


<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [7]:
def month_day(df_multipleyears):
    #your code to reform data here
    df_groupedbymonthday = df_multipleyears.groupby([df_multipleyears.index.month,df_multipleyears.index.day]).agg(min_T=('Tmin', 'min'),max_T=('Tmax', 'max'))
    # df_groupedbymonthday = df_multipleyears.groupby([df_multipleyears.index.month,df_multipleyears.index.day]).agg(min_T=('Tmin', 'min'))
    print(df_groupedbymonthday)

In [8]:
#Test your code
def test_reformed(df):
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day(df)

test_reformed(df)

           min_T  max_T
Date Date              
1    1      -5.8   11.1
     2      -7.5   10.2
     3     -12.6   10.7
     4      -4.1    9.2
     5      -6.0    8.4
...          ...    ...
12   27     -4.8   11.7
     28     -4.2   10.9
     29     -6.7   11.4
     30    -10.2    9.5
     31    -10.6   11.8

[366 rows x 2 columns]


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [12]:
#your code here
#your motivation here
import pandas as pd
import numpy as np
import panel as pn
import os
import re
import yaml
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Band, ColumnDataSource
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
df = df[df.index.year== 2018]

df['Tmin_rolling'] = df['Tmin'].rolling(5).mean()
df2 = df['Tmin']
#df['Tmin_rolling'] = df['Tmin'].rolling(7).mean()
fit1 = SimpleExpSmoothing(df2,initialization_method="heuristic").fit(smoothing_level=0.2, optimized= False)
#expSmoothing = SimpleExpSmoothing(df).fit(smoothing_level=0.1, optimized=False)
fit2 = Holt(df2).fit()

p = figure(sizing_mode="stretch_width",toolbar_location="below" ,x_axis_type="datetime")
p.line(df.index, df["Tmin"],legend_label="Tmin", line_width=2, color="blue", alpha=0.5)
p.line(df.index, df["Tmin_rolling"],legend_label="Tmin_rolling", line_width=2, color="red", alpha=0.5)
p.line(df.index, fit1.fittedvalues,legend_label="Simple Exp Smoothing", line_width=2, color="orange", alpha=0.5)
p.line(df.index, fit2.fittedvalues,legend_label="Holt Smoothing Method", line_width=2, color="yellow", alpha=0.5)
p.legend.location = "top_right"
show(p)


c:\Users\Hamed\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\Hamed\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

In [11]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d
from bokeh.models import Band, ColumnDataSource
import panel as pn
pn.extension()
def drawPlot():
    source = ColumnDataSource(df.reset_index())
    p = figure(sizing_mode="stretch_width",toolbar_location="below" ,x_axis_type="datetime")
    p.line(df.index, df["Tmean"],legend_label="Tmean", line_width=2, color="blue", alpha=0.5)
    p.legend.location = "top_right"

    band = Band(base='Date', lower='Tmin', upper='Tmax', source=source, level='underlay',
            fill_alpha=1.0, line_width=1, line_color='black')
    p.add_layout(band)


    show(p)
drawPlot()

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather